# （1）：基础数据统计
# 内容：读取 Excel/CSV 格式的获奖名单（表 1），统计 C/C++、Python、Java 三个组别各自的一、二、三等奖人数。
# 考查点：pandas 库的使用，数据筛选（Filtering）与分组计数（groupby 或 value_counts）。
count_size=df2.groupby(['组别','奖项']).size().unstack(fill_value=0)

In [4]:
import pandas as pd
import numpy as np
df1=pd.read_excel("res/报名表.xlsx")
df2=pd.read_excel("res/获奖名单.xlsx")
count_size=df2.groupby(['科目组别','奖项']).size().unstack(fill_value=0)
print("各组别各奖项人数统计：")
print(count_size)

各组别各奖项人数统计：
奖项          一等奖  三等奖  二等奖
科目组别                     
C/C++程序设计    15   59   34
Java软件开发      8   22   12
Python程序设计    5   19   22
Web应用开发       2    0    0


### 任务（2）：多表关联与复杂透视
内容：将“获奖名单（表 1）”与“参赛选手基本信息（表 2）”通过“准考证号/姓名”关联，统计各二级学院在不同奖项上的分布情况。
考查点：多表连接（pd.merge）、数据透视表（pivot_table）。

In [ ]:
df3=pd.merge(df1,df2,on='学号',how='left')
print(df3.head())
size_situation=df2.groupby(["院系","科目组别","奖项"]).size().unstack(fill_value=0)
print("各院系各科目组别各奖项人数统计：")
print(size_situation)

           报名编号             证件号码          学号 姓名_x  手机         院系_x  \
0           NaN  340403********7  2023300752  廖林义 NaN    安全科学与工程学院   
1  2.648686e+12  362401********2  2002307152  邓正祥 NaN  空间信息与测绘工程学院   
2  2.648685e+12  342401********X  2018304804  沈先龙 NaN    安全科学与工程学院   
3  2.648686e+12  341502********1  2021300571  郭韦刚 NaN   计算机科学与工程学院   
4  2.648686e+12  340521********4  2021300603   徐超 NaN       土木建筑学院   

           科目    组别   导师          订单编号  缴费方式                     工行卡号  \
0   C/C++程序设计  大学B组   马莹  1.733052e+12  学生缴费      622202********01370   
1  Python程序设计  大学B组  梁载涛  1.733671e+12  学生缴费      622202********30983   
2  Python程序设计  大学B组  曹芮浩  1.733753e+12  院校缴费                      NaN   
3  Python程序设计  大学B组   李声  1.734080e+12  学生缴费      622202********59283   
4   C/C++程序设计  大学B组   方娜  1.733666e+12  学生缴费  6222 0******** 9066 775   

        比赛名称 姓名_y        准考证号         院系_y        科目组别 指导老师   奖项 是否入围国赛  
0        NaN  NaN         NaN          NaN         NaN  NaN  NaN  

: 

任务（3）：非结构化数据解析（进阶）
内容：如果不提供 Excel，只给一个获奖名单的 PDF 扫描件或电子档，要求学生自行提取数据并转化为结构化表格。
考查点：PDF 文本提取库（如 pdfplumber 或 PyMuPDF），正则表达式（re）清洗。

In [44]:
import fitz
import pandas as pd
import re

PDF_PATH = "res/省赛获奖名单.pdf"
OUT_XLSX = "省赛获奖名单.xlsx"

EXAM_ID_RE = re.compile(r'^\d{7,11}$')
AWARD_SET = {"一等奖", "二等奖", "三等奖"}
ADVANCE_SET = {"是", "否"}

rows = []

doc = fitz.open(PDF_PATH)

for page_index, page in enumerate(doc):
    if page_index == 0:
        continue  # 第一页不是正式表格

    blocks = page.get_text("blocks")

    # 收集文本
    lines = []
    for b in blocks:
        if b[6] == 0 and b[4]:
            lines.extend(
                l.strip() for l in b[4].splitlines() if l.strip()
            )

    current = []

    for token in lines:
        # 准考证号 = 一条记录的起点
        if EXAM_ID_RE.match(token):
            if current:
                rows.append(current)
            current = [token]
        else:
            current.append(token)

    if current:
        rows.append(current)

# ===== 结构化 =====
data = []

for r in rows:
    # r: [准考证号, 学校..., 姓名, 科目, 奖项, 是否进决赛]
    if len(r) < 5:
        continue

    exam_id = r[0]
    rest = r[1:]

    # 找奖项
    award_idx = None
    for i, x in enumerate(rest):
        if x in AWARD_SET:
            award_idx = i
            break
    if award_idx is None:
        continue

    award = rest[award_idx]
    advance = (
        rest[award_idx + 1]
        if award_idx + 1 < len(rest) and rest[award_idx + 1] in ADVANCE_SET
        else ""
    )

    front = rest[:award_idx]
    if len(front) < 3:
        continue

    subject = front[-1]
    name = front[-2]
    school = "".join(front[:-2]).strip()

    data.append({
        "省份": "安徽",
        "准考证号": exam_id,
        "学校名称": school,
        "考生姓名": name,
        "科目名称": subject,
        "奖项": award,
        "是否进入决赛": advance
    })

df = pd.DataFrame(data)
df.to_excel(OUT_XLSX, index=False)

print("完成，记录数:", len(df))


完成，记录数: 2268
